---
syncID: 
title: "Stacking and Joining NEON Data with Python"
description: ""
dateCreated: 2020-05-07
authors: Maxwell J. Burner
contributors: Donal O'Leary
estimatedTime: 
packagesLibraries: requests, json, pandas
topics: api, data management, reshaping data
languagesTool: python
dataProduct: DP1.10003.001
code1: 
tutorialSeries: python-neon-api-series
urlTitle: python-neon-api-06-stacking
---

In this tutorial we will learn how to stack and join NEON data tables using the Panda's library.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Combine NEON data tables from different sites and months using Pandas *concat* function
* Describe the difference between inner joins, left outer joins, right outer joins, and full outer joins
* Combine NEON data tables of different types using Pandas *merge* method


### Install Python Packages

* **requests**
* **json** 
* **pandas**


</div>

In this tutorial we will learn how to combine different two or more tables loaded as Python dataframes into one. The NEON API returns data in seperate tables for each month and site. Furthermore, a data product package for one site and month usually includes mutiple tables, related to each other but storing different variables. 

As a result, the data we want for a particular study may be spread across multiple tables. In order to effectively manipulate our data in Python, we will usually want to combine these tables into one. When combining data tables of the same type aross different months or sites, we often call this process *stacking*. When combining tables of different types, we call this *joining*..

Luckily, the Pandas package includes functions and methods for both *stacking* and *joining*  data frames. Here we will discuss stacking data frames using a fairly simple concatenation method, and the more complex subject of joining dataframes. For our examples, we will again use breeding landbird count data from NEON's lower Teakettle site.

In [1]:
import requests
import json
import pandas as pd

In [2]:
SERVER = 'http://data.neonscience.org/api/v0/'
SITECODE = 'TEAK'
PRODUCTCODE = 'DP1.10003.001'

## Stacking tables

As we have seen, NEON stores data of the same type in seperate tables by site and month. But what if we want data spanning multiple months, or from multiple sampling sites? If the data tables are of the same 'type' - such as basic package bird count data - they will have the same columns and headers, so we can simply combine rows from multiple tables into one.

From our first tutorial, we know that bird count data taken at Lower Teakettle is available for several months, including both June and July of 2019. For our example here, we will merge the basic bird count data tables for those two months into one.

In [3]:
#Define a function that takes a 'data/' endpoint json and downloads a csv file from it based on provided strings
#Input: A json with results of a NEON 'data/' endpoint API search, two strings indicating file name
#Output: A pandas dataframe
def get_data(data_json, string1, string2 = ''):
    for file in data_json['data']['files']:
        if(string1 in file['name']):
            if(string2 in file['name']):
                return pd.read_csv(file['url'])
    print('No files matching name')
    return(0)
 

In [4]:
#Request information on data
june_req = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2019-06')
july_req = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2019-07')

june_json = june_req.json()
july_json = july_req.json()

In [5]:
#Read in basic bird count data for June and July 2019 of lower Teakettle
df_june_count = get_data(june_json, 'countdata', 'basic')
df_july_count = get_data(july_json, 'countdata', 'basic')

In [6]:
#View first three rows of June data
df_june_count.head(3)

,uid,namedLocation,domainID,siteID,plotID,plotType,pointID,startDate,eventID,pointCountMinute,...,scientificName,taxonRank,vernacularName,observerDistance,detectionMethod,visualConfirmation,sexOrAge,clusterSize,clusterCode,identifiedBy
0,ddc3e9db-8d75-4447-8d29-4016cce4c3ee,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],5,...,Passerella iliaca,species,Fox Sparrow,108.0,singing,No,Unknown,1.0,NaN,JTIET
1,c1b322de-c2a7-41a3-b9c6-fa5dff60f8d3,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],1,...,Pipilo chlorurus,species,Green-tailed Towhee,58.0,singing,No,Unknown,1.0,NaN,JTIET
2,bbf5d4cb-6622-4f12-a062-5c83b24ce883,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],2,...,Catharus guttatus,species,Hermit Thrush,166.0,singing,No,Unknown,1.0,NaN,JTIET


In [7]:
#View first three rows of July count data
df_july_count.head(3)

,uid,namedLocation,domainID,siteID,plotID,plotType,pointID,startDate,eventID,pointCountMinute,...,scientificName,taxonRank,vernacularName,observerDistance,detectionMethod,visualConfirmation,sexOrAge,clusterSize,clusterCode,identifiedBy
0,9d75150f-ac4f-4510-bc22-f09f1c466fb2,TEAK_008.birdGrid.brd,D17,TEAK,TEAK_008,distributed,A3,2019-07-01T12Z,TEAK_008.A3.2019-07-01T05:21-07:00[US/Pacific],3,...,Turdus migratorius,species,American Robin,139.0,singing,No,Unknown,1.0,NaN,JTIET
1,0b727188-9e5e-466c-9d59-e91feaebd08b,TEAK_008.birdGrid.brd,D17,TEAK,TEAK_008,distributed,A3,2019-07-01T12Z,TEAK_008.A3.2019-07-01T05:21-07:00[US/Pacific],3,...,Piranga ludoviciana,species,Western Tanager,128.0,calling,No,Unknown,1.0,NaN,JTIET
2,bc4ca781-9384-4a86-865a-7183be98b10c,TEAK_008.birdGrid.brd,D17,TEAK,TEAK_008,distributed,A3,2019-07-01T12Z,TEAK_008.A3.2019-07-01T05:21-07:00[US/Pacific],2,...,Cyanocitta stelleri,species,Steller's Jay,119.0,calling,No,Unknown,1.0,NaN,JTIET


In [8]:
#View shape of each count data table
print('June rows and columns: ',df_june_count.shape)
print('July rows and columns: ',df_july_count.shape)

June rows and columns:  (1484, 22)
July rows and columns:  (286, 22)


The two dataframes have matching number of columns, column names, and column data types. We can directly combine them into one child table containing the rows of both parent tables using the Pandas *concat* function. This takes a list, tuple, or other sequence of data frames, and tries to combine them into one.

The *concat* function can combine rows, concatenating along the index, or it can combine columns, concatenating along the header. Which approach is used is determined by the *axis* parameter; 0 for stacking rows, 1 for stacking columns. The default is 0, so in this case we don't have to specify.

In [11]:
#Combine data frames along index
df_concatenated = pd.concat((df_june_count, df_july_count))
print(df_concatenated.shape)

(1770, 22)


The new dataframe has 1770 rows, the sum of the 1484 rows from the June data and the 286 rows from the July data.

## Joining Tables

At other times, the data we want may be spread across two or more related tables, each with different attributes. In this case we will want to **join** the tables together. Joining tables is an important concept in data sciene and analysis. Several different kinds of joins exist, depending on the relationship between the tables, how we want to handle unmatched data. 

The breeding land bird count data comes with several tables, including the 'Count Data' table (which contains all of the different individual bird observations and their taxonomic information) and the 'Point Data' table (which contains information about the point where the birds were observed (spatial information, vegetation type, etc.) as well as the observation conditions for the day (cloud cover, wind speed, relative humidity, etc.). In order to match up every bir observation with the location and conditions in which it was observed, we will need to join these two tables together.

For this example, we will download the per point data for June as a dataframe, and join it with the count data for June. 

In [12]:
#Import basic package per point data into Python
df_june_point = get_data(june_json, 'perpoint', 'basic')

#View shape of new dataframe
print(df_june_point.shape)

(81, 27)


In [13]:
#View columns of count data table
print('Count data')
print(df_june_count.dtypes)

print('\nPoint Data')
print(df_june_point.dtypes)

Count data
uid                    object
namedLocation          object
domainID               object
siteID                 object
plotID                 object
plotType               object
pointID                object
startDate              object
eventID                object
pointCountMinute        int64
targetTaxaPresent      object
taxonID                object
scientificName         object
taxonRank              object
vernacularName         object
observerDistance      float64
detectionMethod        object
visualConfirmation     object
sexOrAge               object
clusterSize           float64
clusterCode            object
identifiedBy           object
dtype: object

Point Data
uid                            object
namedLocation                  object
domainID                       object
siteID                         object
plotID                         object
plotType                       object
pointID                        object
nlcdClass                      object

Notice that the countdata table and the perpoint table share a number of common columns/variables/attributes. We say that a *relationship* exists between these tables; data in a certain column of one table corresponds to data in a certain column of the other table. To join two tables, we select a pair of related columns from the tables to serve as the *keys* in the join.

One of the columns present in both tables is 'eventID'. Do the values of this column in one table appear to correspond to those in the other table?

In [14]:
#Print first ten rows of eventID column for each table
for i in range(10):
    print('Point: ',df_june_count['eventID'][i],'\tCount: ' ,df_june_point['eventID'][i])

Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.A3.2019-06-10T06:03-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.A2.2019-06-10T06:20-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.B2.2019-06-10T06:36-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.A1.2019-06-10T06:53-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.B1.2019-06-10T07:09-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.C1.2019-06-10T07:25-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_003.C2.2019-06-10T07:40-07:00[US/Pacific]
Point:  TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific] 	Count:  TEAK_031.C1.2019-06-10T07:54-07:00[US/Pacific]
P

It appears that it should be possible to match values between the two eventID columns; in fact, the two columns have idetnical first entries.

If we are using just one key pair, we ideally want there to be no repeated values in the  key column of the table with fewer rows. This way, every row in the longer table matches to no more than one row in the shorter table.

Here the shorter table is the perpoint data; let's see if the eventID attribute has only unique values.

In [15]:
#Compare shape of whole eventID column to shape of eventID column after removing repeated values.
print(df_june_point['eventID'].shape)
print(pd.unique(df_june_point['eventID']).shape)


(81,)
(81,)


It looks like there are no repeat values in the eventID column of the per point table. Combined with the fact that the eventID values in the count data table correspond to eventID values in the  can use this as a key. Our join will use every row of the longer count data table only once, and but will duplicate rows of the shorter per point data table as necessary so that the number of rows match. Our final table will have the same number of rows as the per point table.

The next question is what kind of join we want; this determines how the join algorithm deals with rows in one table that lack a matching row in the other. If we perform an **Inner Join**, any rows from either table that lack a match in the other will be left out of the final table; if all but three rows of the longer table have matching rows in the shorter table, then final table will have three rows fewer than the longer parent table.

![Inner Join Diagram](https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png)

If we instead perform an **Outer Join**, then rows that lack a match will be allowed to stay in the final table; the added columns will be "padded" with NA values in the output table. If we only keep unmatched row from the left table it is a *Left Outer Join* or **Left Join**; if we only keep unmatched rows from the right table it is a *Right Outer Join* or **Right Join**; and if we keep unmatched rows from both tables it is a *Full Outer Join* or **Full Join**.

![Outer Join Diagram](https://d33wubrfki0l68.cloudfront.net/9c12ca9e12ed26a7c5d2aa08e36d2ac4fb593f1e/79980/diagrams/join-outer.png)

In this example we will do an inner join, as we don't want any rows with missing data. The final table will have the same number of rows as the count data table, the longer table, minus any rows that didn't have a match in the point data table.


The next question is which columns from each table we want to include in the final table. The per point data table has quite a few columns that are related to columns in the count data table; bringing these along would add redundant data.

In [16]:
#View columns of June count data table
print(df_june_count.dtypes)

uid                    object
namedLocation          object
domainID               object
siteID                 object
plotID                 object
plotType               object
pointID                object
startDate              object
eventID                object
pointCountMinute        int64
targetTaxaPresent      object
taxonID                object
scientificName         object
taxonRank              object
vernacularName         object
observerDistance      float64
detectionMethod        object
visualConfirmation     object
sexOrAge               object
clusterSize           float64
clusterCode            object
identifiedBy           object
dtype: object


In [17]:
#View columns of June per point data table
df_june_point.dtypes

uid                            object
namedLocation                  object
domainID                       object
siteID                         object
plotID                         object
plotType                       object
pointID                        object
nlcdClass                      object
decimalLatitude               float64
decimalLongitude              float64
geodeticDatum                  object
coordinateUncertainty         float64
elevation                     float64
elevationUncertainty          float64
startDate                      object
eventID                        object
startCloudCoverPercentage       int64
endCloudCoverPercentage         int64
startRH                         int64
endRH                           int64
observedHabitat                object
observedAirTemp                 int64
kmPerHourObservedWindSpeed      int64
laboratoryName                 object
samplingProtocolVersion        object
remarks                       float64
measuredBy  

The two dataframes share the columns 'namedLocation', 'domainID', 'siteID', 'plotID', 'plotType', 'pointID', 'startDate', and 'eventID'. Both tables also have a 'uid' column, but these are just values used to index each row in a table; the 'uid' values in the count data table are not related to the 'uid' values in the point data table.

Our merge will bring over values from all of the point data table column that aren't already present in the point data table. We also won't bring over 'uid' values from the point data table; these would not be of any use in the merged table.



In [18]:
#Create a list of columns to keep; columns only present in the point data, plus eventID
good_columns = []
for column in df_june_point:
    if(column not in df_june_count.columns):
        good_columns.append(column)
        
good_columns.append('eventID')
print(good_columns)

['nlcdClass', 'decimalLatitude', 'decimalLongitude', 'geodeticDatum', 'coordinateUncertainty', 'elevation', 'elevationUncertainty', 'startCloudCoverPercentage', 'endCloudCoverPercentage', 'startRH', 'endRH', 'observedHabitat', 'observedAirTemp', 'kmPerHourObservedWindSpeed', 'laboratoryName', 'samplingProtocolVersion', 'remarks', 'measuredBy', 'eventID']


The Pandas package offers a couple of different methods for joining data frames. In this case we will use the **.merge** method of dataframes. This method is called on one 'left' dataframe, and is passed another 'right' dataframe, along with further parameters indicating how the join is to occur: *left_on* and *right_on* indicate the column from the left and right dataframes to be used as keys respectively, and the *how* parameter is given a string 'inner' (default), 'left', 'right', or 'full' that determines the type of join performed.

You can read the Pandas documentation for *pandas.Dataframe.merge* [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) (pandas.pydata.org). Pandas also offers a *.join* method for the same purpose; that method has fewer parameters, but assumes that the key column in the right table is that table's index.

Now we make the join. First we will use the *.filter* method and the list we prepared to get a version of the point data table that only contains the desired columns. Then we call *.merge* on the count data, and pass the prepared data table.

In [19]:
#Prepare point dataframe for joining by extracting the desired columns
df_right = df_june_point.filter(items = good_columns, axis = 1)

In [20]:
#Join data frames
df_joined = df_june_count.merge(df_right, how = 'inner', left_on = 'eventID', right_on = 'eventID')
print(df_joined.columns)

Index(['uid', 'namedLocation', 'domainID', 'siteID', 'plotID', 'plotType',
       'pointID', 'startDate', 'eventID', 'pointCountMinute',
       'targetTaxaPresent', 'taxonID', 'scientificName', 'taxonRank',
       'vernacularName', 'observerDistance', 'detectionMethod',
       'visualConfirmation', 'sexOrAge', 'clusterSize', 'clusterCode',
       'identifiedBy', 'nlcdClass', 'decimalLatitude', 'decimalLongitude',
       'geodeticDatum', 'coordinateUncertainty', 'elevation',
       'elevationUncertainty', 'startCloudCoverPercentage',
       'endCloudCoverPercentage', 'startRH', 'endRH', 'observedHabitat',
       'observedAirTemp', 'kmPerHourObservedWindSpeed', 'laboratoryName',
       'samplingProtocolVersion', 'remarks', 'measuredBy'],
      dtype='object')


Finally, let's check the shape of the joined data table against the original count data table.

In [21]:
print(df_june_count.shape)
print(df_joined.shape)

(1484, 22)
(1484, 40)


The number of rows is the same in each, so it appears that every entry in the count data was successfully mapped to an entry in the point data.

Finally, let's take a look at the new joined data.frame to check our work.

In [23]:
pd.set_option('max_columns', 40) #show all columns
df_joined.head(3)

,uid,namedLocation,domainID,siteID,plotID,plotType,pointID,startDate,eventID,pointCountMinute,targetTaxaPresent,taxonID,scientificName,taxonRank,vernacularName,observerDistance,detectionMethod,visualConfirmation,sexOrAge,clusterSize,clusterCode,identifiedBy,nlcdClass,decimalLatitude,decimalLongitude,geodeticDatum,coordinateUncertainty,elevation,elevationUncertainty,startCloudCoverPercentage,endCloudCoverPercentage,startRH,endRH,observedHabitat,observedAirTemp,kmPerHourObservedWindSpeed,laboratoryName,samplingProtocolVersion,remarks,measuredBy
0,ddc3e9db-8d75-4447-8d29-4016cce4c3ee,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],5,Y,FOSP,Passerella iliaca,species,Fox Sparrow,108.0,singing,No,Unknown,1.0,NaN,JTIET,evergreenForest,37.012747,-119.009359,WGS84,250.6,2190.7,1.1,40,50,79,76,evergreen forest,7,2,Bird Conservancy of the Rockies,NEON.DOC.014041vJ,NaN,JTIET
1,c1b322de-c2a7-41a3-b9c6-fa5dff60f8d3,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],1,Y,GTTO,Pipilo chlorurus,species,Green-tailed Towhee,58.0,singing,No,Unknown,1.0,NaN,JTIET,evergreenForest,37.012747,-119.009359,WGS84,250.6,2190.7,1.1,40,50,79,76,evergreen forest,7,2,Bird Conservancy of the Rockies,NEON.DOC.014041vJ,NaN,JTIET
2,bbf5d4cb-6622-4f12-a062-5c83b24ce883,TEAK_003.birdGrid.brd,D17,TEAK,TEAK_003,distributed,B3,2019-06-10T13Z,TEAK_003.B3.2019-06-10T05:45-07:00[US/Pacific],2,Y,HETH,Catharus guttatus,species,Hermit Thrush,166.0,singing,No,Unknown,1.0,NaN,JTIET,evergreenForest,37.012747,-119.009359,WGS84,250.6,2190.7,1.1,40,50,79,76,evergreen forest,7,2,Bird Conservancy of the Rockies,NEON.DOC.014041vJ,NaN,JTIET


We see that now every individual bird observation is listed with the associated location and observation conditions; just as we wanted.

If you want to learn more about this subject, the Pandas package's official site offers an in-depth [discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) (pandas.pydata.org) of using *concat*, *join*, *merge*, and other functions to combine data frames.